# mlworkflow tutorial

This tutorial is not meant to show every possible use or functionality of the mlworkflow library. It is rather to expose some of them that fit in an interactive context so that people may see if some of the features can be useful to them.

Our goal is at the moment simply to see if our work could benefit other people. Feedbacks are of course welcome.

## What is mlworkflow?

It is a library providing several separate modules to:

- Have an interactive feedback from your model being trained
- Aggregate data about your models and experiments, accessing them in a convenient manner
- Help you to obtain replayable and modifiable experiments
- Put notes and comments on them
- Make the use of your models more practical

That the modules are there do not mean you should use them. Only use them if they fit your way of experimenting. If you have other habits that you feel like would deserve a place in the library, we strongly encourage you to try to implement it in a generic fashion and to submit a pull-request, or to suggest it.

This library is developped at UCLouvain, and is mostly an attempt to make people feel that "tensorboard" does not answer to everything we may want about keeping tracks of our experiments. It is under MIT license.

## What is not currently handled that could frustrate users?

The interactive parts of the library (LivePanels, Dashboard) mostly rely on the use of Jupyter notebooks. However, a lot of people do not run their experiments in Jupyter notebooks. (Here, we use them as "front-ends" to our models, ...) So you may want to create summaries from Python scripts as well and only run the dashboard in a notebook. This problem will most likely be addressed. For the rest (Dataset, DataCollection, ...) nothing should rely on Jupyter. If it does, this should be fixed.

There is not much documentation and not all the features are commented yet, but we hope the tutorial may help you to get started with it, or why not getting you to ask for more.

Finally, remember this is best described as a side-project for accelerating our research that we thought may benefit other people.

In [ ]:
# Some nice code you may want to have in a file for versioning, ...
from mlworkflow.datasets import DictDataset, TransformedDataset
from mlworkflow.tf_utils import TFModel
import tensorflow as tf
import numpy as np

def graph_definition(additional_layers=[], optimizer=tf.train.RMSPropOptimizer):
    """Some graph definition, could be anything!"""
    x = inputs = tf.placeholder(tf.float32, [None, 28, 28, 1])
    
    with tf.variable_scope("shared"):
        x = tf.layers.conv2d(x, 64, kernel_size=[3,3], strides=[2,2], padding="same", activation=tf.nn.relu)
        x = tf.layers.conv2d(x, 128, kernel_size=[3,3], strides=[2,2], padding="same", activation=tf.nn.relu)
        x = tf.layers.conv2d(x, 256, kernel_size=[3,3], strides=[2,2], padding="same", activation=tf.nn.relu)
    
    x = tf.layers.flatten(x)
    
    for layer_defn in additional_layers:
        x = layer_defn(x)
    
    x = logits = tf.layers.dense(x, units=10)
    
    y_pred = tf.nn.softmax(x)
    y_true = tf.placeholder(tf.float32, [None, 10])
    
    training = tf.placeholder(tf.bool)  # Would only be useful for batchnorm or dropout
    learning_rate = tf.Variable(1e-3)
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=logits)
    loss = tf.reduce_sum(loss, axis=0)
    
    optimizer = optimizer(learning_rate)
    step = optimizer.minimize(loss)
    
    # "inputs", "outputs", "targets", "loss", "step" and "training" are necessary
    # for TFModel.train, TFModel.predict and TFModel.evaluate require less,
    # but this is meant to be easily modifiable to suit your needs.
    return dict(inputs=inputs,
                outputs=y_pred,
                targets=y_true,
                lr=learning_rate,
                logits=logits,
                loss=loss,
                step=step,
                training=training)


def dataset_definition(return_keys=False):
    # We create a mlworkflow.dataset DictDataset, but this is just for my convenience
    # and later being able to quickly integrate data augmentation, ...
    # There are much more powerful tools than DictDataset... (see PickledDataset,
    # pickle_or_load, TransformedDataset, AugmentedDataset...)
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("tutorial_files/MNIST/", one_hot=True)
    keys, dic = {}, {}
    
    dic.update({i: item
                for i, item in enumerate(zip(mnist.train.images,
                                             mnist.train.labels))})
    keys["training_keys"] = list(range(len(dic)))
    
    old_size = len(dic)
    dic.update({i: item
                for i, item in enumerate(zip(mnist.validation.images,
                                             mnist.validation.labels),
                                         old_size)})
    keys["validation_keys"] = list(range(old_size, len(dic)))
    
    old_size = len(dic)
    dic.update({i: item
                for i, item in enumerate(zip(mnist.test.images,
                                             mnist.test.labels),
                                             old_size)})
    keys["testing_keys"] = list(range(old_size, len(dic)))
    
    dataset = DictDataset(dic)
    dataset = TransformedDataset(dataset)
    @dataset.add_transform
    def reshape(item):
        x, y = item
        x = np.reshape(x, [28, 28, 1])
        return x, y
    
    if return_keys:
        return dataset, keys
    return dataset

# Experimenting

We first create our model and dataset

In [ ]:
from mlworkflow import DataCollection, Call
# We create a persistent representation of how our model and dataset are instantiated,
# and instantiate them, as well as log some other data
# Notice all of this is framework-agnostic! (except what comes from mlworkflow.tf_utils)

data = DataCollection("tutorial_files/experiment_{}.dcp")
data["model"] = Call(TFModel)(initializer=Call(graph_definition)(
    # Let's not add a single layer for now
    additional_layers=[]  # Empty list by default
).partial())
print("A picklable and replayable representation of the creation of the model:\n", data["model"])
print("Notice it will look for 'graph_definition' in the module it taken it from.\n")
data["dataset"] = Call(dataset_definition)
# You may also want to store the code of some file, whatever, you can put anything
# that can be pickled in "data"

model = data["model"].eval()
dataset, keys = data["dataset"](return_keys=True).eval()

globals().update(keys)
data.update(keys)

data.filename

In [ ]:
from mlworkflow import run_in_cell
from mlworkflow import LivePanels, SideRunner

from matplotlib import pyplot as plt
%matplotlib inline
from IPython import display
import random

@run_in_cell  # This will run the body of this function as well as let us reuse it later
def train(model, data, epochs=20):
    # To run data augmentation in parallel
    sd = SideRunner()
    # To have progress bar and updateable sub-outputs
    panels = LivePanels(["head", "batch_prog", "body", "val_perf"], record=["body"])

    for epoch in panels.tqdm_notebook(range(epochs)):
        data[["training_error", "validation_error"]] = 0, 0
        # randomize batch composition
        random.shuffle(training_keys);random.shuffle(validation_keys)
        n = 0
        for x, y in sd.yield_async(dataset.batches(training_keys, 2048)):
            data["training_error"] += model.train(x, y)
            n += x.shape[0]
            panels["batch_prog"] = "Training {}/{}".format(n, len(training_keys))
        data["training_error"] /= n

        n = 0
        for x, y in sd.yield_async(dataset.batches(validation_keys, 2048)):
            data["validation_error"] += model.train(x, y)
            n += x.shape[0]
            panels["batch_prog"] = "Validating {}/{}".format(n, len(validation_keys))
        data["validation_error"] /= n

        display.clear_output()
        # History WITH CURRENT VALUES APPENDED (only because of the "_" of "history_"
        plt.plot(data.history_[:,["training_error", "validation_error"]])
        plt.show()
        # We could also show some examples

        data.save_external("weights", model.get_variables())

        data["recording"] = panels.current_record
        data.checkpoint()
    return "some_return"
assert train.result == "some_return"

In [ ]:
from mlworkflow import ListFromArgs
restore_weights = False
new_data, data = DataCollection.create_with_parent("tutorial_files/experiment_{}.dcp",
                                                   parent=data.filename)
# Restore everything from that experiment, we'll only change the model
new_data.update(data[-1])
# Just change the optimizer, add some layers
new_data["model"] = data[-1,"model"](
    initializer=data[-1,"model"]["initializer"](
        optimizer=Call("tensorflow", "train.AdamOptimizer").partial(),  # Some hand-made reference
        # Creates a list with children that have to be evaluated
        additional_layers=Call(ListFromArgs).with_args(  
            Call(tf.layers.dense)(units=100, activation=tf.nn.relu).partial()
        )
    )
)
print(new_data["model"])
print("Please note that the activation is picklable (because accessible globally), but does not have "
      "a reproducible string. We could wrap it into Call(...).partial()")
new_model = new_data["model"].eval()
# We could even restore the weights from the common parts
if restore_weights:
    new_model.set_variables({k:v for k, v in data.external[-1,"weights"]
                             if k.startswith("shared/")})
# Reuse the "train" function from
train(new_model, new_data, epochs=10)

# Compare both experiments

In [ ]:
from mlworkflow import find_files
# Take the last two files
runs = DataCollection.load_files(find_files("tutorial_files/*.dcp")[-2:])
# Compare last 10 training errors
print("Training error")
plt.plot(np.array(runs[:,-10:,"training_error"]).T)
plt.show()
# plot validation_error
print("Validation error")
for run in runs.values():
    plt.plot(run[:,"validation_error"])
plt.show()

# Dashboards

In [ ]:
from mlworkflow import dashboard as d
d.Dashboard([[d.list_files(path="tutorial_files/*.dcp", reverse=True),
              d.vbox(d.tags(), d.comment())  # Tags and comments in metadata
             ],
             [d.filename(link_parent=True)],  # Show filename and link to parent if any
             [d.list_properties()],  # List fields of data and meta_data
             [d.recording()],  # Replay outputs recorded by panels into data["recording"]
             [d.execute(globals())],  # Plays some code in ```code sections``` in the comment metadata
            ])